In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import gridspec

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Have a look to a sample time serie
Set No. 2, first file:
- 2kHz acquisition rate, 1s in total

In [ ]:
sample2 = pd.read_csv("/kaggle/input/nasa-bearing-dataset-aggregated-sets-no-1-2-3/sample_2nd.csv",index_col=0)
sample2.shape

In [ ]:
figure(figsize=(15, 6), dpi=80)
sample2['Bearing 1'].loc[:1000].plot()
plt.show()

# Dataset preprocessing

In [ ]:
# Read the CSV file and set first column as the dataframe index
dataset = pd.read_csv("/kaggle/input/nasa-bearing-dataset-aggregated-sets-no-1-2-3/merged_dataset_BearingTest_2.csv",
                      index_col=0)
dataset.describe()

## Normalize the dataset

In [ ]:
from sklearn import preprocessing

# Dataset is scaled so that maximum for every column is 1
scaler = preprocessing.MinMaxScaler()
dataset_scaled = pd.DataFrame(scaler.fit_transform(dataset), 
                              columns=dataset.columns, 
                              index=dataset.index)
dataset_scaled.describe()

In [ ]:
x_ticks_span = 50

dataset_scaled.plot(figsize=(18, 4))
plt.xlabel('Timestamp')
plt.xticks(np.arange(0, dataset_scaled.shape[0], x_ticks_span), fontsize=10, rotation = 30)
plt.ylabel('Acceleration')


plt.legend(loc="upper left")  
plt.title('Time series for all 4 accelerometers (normalized signals)', fontweight ="bold")
plt.show()

## Build train and test datasets
- We want the training set contains only "normal" data
- The rest of points will be in the test set, that will contain both "normal" and anomalous data

In [ ]:
print("dataset_scaled shape is",dataset_scaled.shape,"\n\n", dataset_scaled.index)

In [ ]:
dataset_train = dataset_scaled[:'2004-02-13 23:52:39']
dataset_test  = dataset_scaled['2004-02-14 00:02:39':]
# Random shuffle training data
dataset_train.sample(frac=1)

print("Train dataset has lenght", dataset_train.shape[0], "while test dataset is", dataset_test.shape[0],
      "TOTAL=", dataset_train.shape[0]+dataset_test.shape[0])

In [ ]:
x_ticks_span = 50

dataset_train.plot(figsize = (6,6), title ='Left time series with "normal" data (normalized signals)')
plt.xticks(np.arange(0, dataset_train.shape[0], x_ticks_span), fontsize=10, rotation = 30)
plt.ylim(0,1)
plt.legend(loc="upper left")  
plt.show()

dataset_test.plot(figsize = (18,6), title='Right time series with "normal" & "anomalous" data (normalized signals)')
plt.xticks(np.arange(0, dataset_test.shape[0], x_ticks_span), fontsize=10, rotation = 30)
plt.ylim(0,1)
plt.legend(loc="upper left")  
plt.show()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18, 5))
fig.suptitle('TRAINING SET: Comparison against Bearing 1', fontsize=20)

axes[0].scatter(np.array(dataset_train['Bearing 1']), np.array(dataset_train['Bearing 2']))
axes[0].set_xlabel('Bearing 1')
axes[0].set_ylabel('Bearing 2')
axes[0].set_title('Bearing 1 vs. 2')
axes[0].set_xlim(0,1)
axes[0].set_ylim(0,1)

axes[1].scatter(np.array(dataset_train['Bearing 1']), np.array(dataset_train['Bearing 3']))
axes[1].set_xlabel('Bearing 1')
axes[1].set_ylabel('Bearing 3')
axes[1].set_title('Bearings 1 vs. 3')
axes[1].set_xlim(0,1)
axes[1].set_ylim(0,1)

axes[2].scatter(np.array(dataset_train['Bearing 1']), np.array(dataset_train['Bearing 3']))
axes[2].set_xlabel('Bearing 1')
axes[2].set_ylabel('Bearing 4')
axes[2].set_title('Bearings 1 vs. 4')
axes[2].set_xlim(0,1)
axes[2].set_ylim(0,1)

plt.show()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(15, 5))
fig.suptitle('TEST SET: Comparison against Bearing 1', fontsize=20)

axes[0].scatter(np.array(dataset_test['Bearing 1']), np.array(dataset_test['Bearing 2']))
axes[0].set_xlabel('Bearing 1')
axes[0].set_ylabel('Bearing 2')
axes[0].set_xlim(0,1)
axes[0].set_ylim(0,1)
axes[0].set_title('Bearing 1 vs. 2')

axes[1].scatter(np.array(dataset_test['Bearing 1']), np.array(dataset_test['Bearing 3']))
axes[1].set_xlabel('Bearing 1')
axes[1].set_ylabel('Bearing 3')
axes[1].set_xlim(0,1)
axes[1].set_ylim(0,1)
axes[1].set_title('Bearings 1 vs. 3')

axes[2].scatter(np.array(dataset_test['Bearing 1']), np.array(dataset_test['Bearing 4']))
axes[2].set_xlabel('Bearing 1')
axes[2].set_ylabel('Bearing 4')
axes[2].set_xlim(0,1)
axes[2].set_ylim(0,1)
axes[2].set_title('Bearings 1 vs. 4')

plt.show()

# PCA model: Principal Components analysis
Apply dimensionality reduction to scale down from 4 dimensions to only 2 signal

In [ ]:
from sklearn.decomposition import PCA

n_components = 4 # How many dimensions you want to reduce to
pca = PCA(n_components=n_components, svd_solver= 'full')

In [ ]:
# Compute all PCA components FOR THE TRAINING SET
X_train_PCA = pca.fit_transform(dataset_train)
X_train_PCA = pd.DataFrame(X_train_PCA)
X_train_PCA.index = dataset_train.index

# Project the TEST SET onto the PCA space
X_test_PCA = pca.transform(dataset_test)
X_test_PCA = pd.DataFrame(X_test_PCA)
X_test_PCA.index = dataset_test.index

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12, 5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])
fig.suptitle('TRAINING & TEST datasets in PCA axes', fontsize=20)

ax0 = plt.subplot(gs[0])
ax0.scatter(X_train_PCA.loc[:,0], X_train_PCA.loc[:,1])
ax0.set_xlabel('PC1')
ax0.set_ylabel('PC2')
ax0.set_xlim(-0.1,0.1)
ax0.set_ylim(-0.1,0.1)
ax0.set_title('Training set Principal Components')

ax1 = plt.subplot(gs[1])
ax1.scatter(X_test_PCA.loc[:,0], X_test_PCA.loc[:,1])
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('Test set Principal Components')

plt.show()

## Variance ratio (as per training set)
We can see that in the PCA space, the variance is maximized along PC1 (explains 93% of the variance) and PC2 (explains 5.5%)
- Components are listed per importance (greater variance contribution)

In [ ]:
np.set_printoptions(precision=3, suppress=True) # 3 decimal places and don't use scientific notation

pca.fit_transform(dataset_train)
print(pca.explained_variance_ratio_)

In [ ]:
np.set_printoptions(precision=1, suppress=False) # Use scientific notation
print(pca.explained_variance_)

### As per the result above, components 1 and 2 accounts for the 76.8 + 14.1 = 90.9% of the total variance

## Reduce the analysis to the first two PCA components

In [ ]:
pca = PCA(n_components= 2, svd_solver= 'full')

# Compute (2) PCA most relevant components FOR THE TRAINING SET
X_train_PCA = pca.fit_transform(dataset_train)
X_train_PCA = pd.DataFrame(X_train_PCA)
X_train_PCA.index = dataset_train.index

# Project the TEST SET onto the PCA space (2 dimensions)
X_test_PCA = pca.transform(dataset_test)
X_test_PCA = pd.DataFrame(X_test_PCA)
X_test_PCA.index = dataset_test.index

# Computing the Mahalanobis distance
The Mahalanobis distance is widely used in cluster analysis and classification techniques. In order to use the Mahalanobis distance to classify a test point as belonging to one of N classes, one first estimates the covariance matrix of each class, usually based on samples known to belong to each class.

In our case, as we are only interested in classifying “normal” vs “anomaly”, we use training data that only contains normal operating conditions to calculate the covariance matrix.
  - This explains while we built the training dataset as explained above

### Functions' definition

In [ ]:
# CALCULATE THE COVARIANCE
def cov_matrix(data, verbose=False):
    covariance_matrix = np.cov(data, rowvar=False)
    if is_pos_def(covariance_matrix):
        inv_covariance_matrix = np.linalg.inv(covariance_matrix)
        if is_pos_def(inv_covariance_matrix):
            return covariance_matrix, inv_covariance_matrix
        else:
            print("Error: Inverse of Covariance Matrix is not positive definite!")
    else:
        print("Error: Covariance Matrix is not positive definite!")
        
# CALCULATE THE MAHALANOBIS DISTANCE
def MahalanobisDist(inv_cov_matrix, mean_distr, data, verbose=False):
    inv_covariance_matrix = inv_cov_matrix
    vars_mean = mean_distr
    diff = data - vars_mean
    md = []
    for i in range(len(diff)):
        md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))
    return md

# CHECK IF MATRIX IS POSITIVE DEFINITE
def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False

## Set up PCA model
Define train/test set from the two main principal components:

In [ ]:
data_train = np.array(X_train_PCA.values)
data_test = np.array(X_test_PCA.values)

Calculate the covariance matrix and its inverse, based on data in the training set:

In [ ]:
cov_matrix, inv_cov_matrix  = cov_matrix(data_train)

We also calculate the mean value for the input variables in the training set, as this is used later to calculate the Mahalanobis distance to datapoints in the test set:

In [ ]:
# Mean of each column: PCA1, PCA2
## - It should be very close to zero
mean_distr = data_train.mean(axis=0) # axis=0 means that average is computed per column
np.set_printoptions(precision=3, suppress=False)
mean_distr

Using the covariance matrix and its inverse, we can calculate the Mahalanobis distance for the training data defining “normal conditions”, and find the threshold value to flag datapoints as an anomaly.
Then calculate the Mahalanobis distance for the datapoints in the test set, and compare that with the anomaly threshold:

In [ ]:
dist_test = MahalanobisDist(inv_cov_matrix, mean_distr, data_test, verbose=False)
dist_train = MahalanobisDist(inv_cov_matrix, mean_distr, data_train, verbose=False)

print("Minimum & maximum MD in training set:", min(dist_train), max(dist_train) )
print("Minimum & maximum MD in test set    :", min(dist_test), max(dist_test) )

In [ ]:
figure(figsize=(15, 4), dpi=80)
plt.plot(np.array(dist_train) , label="Baseline (25% of dataset, i.e. first 225 points)")
plt.plot(np.array(dist_test)[:600], label="Analysis data (first 600 out of 759 points)")
plt.legend(loc="upper left")
plt.title("Comparison of test set (analysis set) distance over train set (baseline)")
plt.show()

You can see that there is a large portion of *normal* data in the analysis (test) set. This validates the lenght of the dataset we selected for establishing the baseline (train set).

## Threshold value for flagging an anomaly

In [ ]:
# CALCULATE THRESHOLD FOR CLASSIFYING AS ANOMALY
def MD_threshold(dist, extreme=False, verbose=False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    return threshold

threshold = MD_threshold(dist_train, extreme = True)
#  * extreme = True => twice the mean of incoming data (dist_train)
#  * extreme = False => three times the mean
print("Threshold value for flagging an anomaly is", "{:.2f}".format(threshold) )

The square of the Mahalanobis distance to the centroid of the distribution should follow a χ2 distribution if the assumption of normal distributed input variables is fulfilled. This is also the assumption behind the above calculation of the “threshold value” for flagging an anomaly. As this assumption is not necessarily fulfilled in our case, it is beneficial to visualize the distribution of the Mahalanobis distance to set a good threshold value for flagging anomalies.

In [ ]:
import seaborn as sns
plt.figure()
sns.distplot(dist_train,
             bins = 20, 
             kde= True, 
            color = 'green');
plt.xlim([0.0,5])
plt.xlabel('Mahalanobis dist')
plt.title('Mahalanobis distance distribution for the baseline (train set)')
plt.show()

From the above distributions, the calculated threshold value of 3.66 for flagging an anomaly seems reasonable (defined as 3 standard deviations from the center of the distribution).

We can then save the Mahalanobis distance, as well as the threshold value and “anomaly flag” variable for both train and test data in a dataframe:

### Outliers in the baseline (*train set*)

In [ ]:
anomaly_train = pd.DataFrame()
anomaly_train['Mob dist']= dist_train
anomaly_train['Thresh'] = threshold
# If Mob dist above threshold: Flag as anomaly
anomaly_train['Anomaly'] = anomaly_train['Mob dist'] > anomaly_train['Thresh']
anomaly_train.index = X_train_PCA.index

n_outliers_train = anomaly_train[ anomaly_train['Anomaly'] == True].shape[0]
print("There are", n_outliers_train, "anomalies in the train set out of", anomaly_train.shape[0], "points")
anomaly_train.head()

In [ ]:
anomaly_train.plot(logy=True, figsize = (15,6), ylim = [1e-1,1e3], color = ['green','red'])
plt.xticks(np.arange(0, anomaly_train.shape[0], 50), fontsize=10, rotation = 30)
plt.title('Baseline plot against anomaly threshold')
plt.show()

### Outliers in the analysis set (*test set*)

In [ ]:
anomaly = pd.DataFrame()
anomaly['Mob dist']= dist_test
anomaly['Thresh'] = threshold
# If Mob dist above threshold: Flag as anomaly
anomaly['Anomaly'] = anomaly['Mob dist'] > anomaly['Thresh']
anomaly.index = X_test_PCA.index

n_outliers = anomaly[ anomaly['Anomaly'] == True].shape[0]
print("There are", n_outliers, "anomalies in the test set out of", anomaly.shape[0], "points")
anomaly_train.head()

In [ ]:
anomaly = pd.DataFrame()
anomaly['Mob dist']= dist_test
anomaly['Thresh'] = threshold
# If Mob dist above threshold: Flag as anomaly
anomaly['Anomaly'] = anomaly['Mob dist'] > anomaly['Thresh']
anomaly.index = X_test_PCA.index
anomaly.head()

Based on the calculated statistics, any distance above the threshold value will be flagged as an anomaly.

We can now merge the data in a single dataframe and save it as a .csv file:

In [ ]:
anomaly_alldata = pd.concat([anomaly_train, anomaly])
anomaly_alldata.to_csv('Anomaly_distance.csv')

We can now plot the calculated anomaly metric (Mob dist), and check when it crosses the anomaly threshold (note the logarithmic y-axis).

In [ ]:
anomaly_alldata.plot(logy=True, figsize = (15,6), ylim = [1e-1,1e3], color = ['green','red'])
plt.xticks(np.arange(0, anomaly_alldata.shape[0], 50), fontsize=10, rotation = 30)
plt.title('Whole dataset plot against anomaly threshold')
plt.show()